In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from spectools import lpsd
lpsd.use_c_core = False

import multiprocessing
pool = multiprocessing.Pool()

In [ ]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real
    
def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)

In [ ]:
N_values = np.logspace(3,5,10).astype(int)
inputs = []
for N in N_values:
    inputs.append(200*band_limited_noise(200, 500, N, 2000))

cpus = [1, 2, 4, 8]
times = {cpu: [] for cpu in cpus}

for cpu in cpus:
    print("Computing with CPU = {}...".format(cpu))
    pool = multiprocessing.Pool(cpu)

    for i, x in enumerate(inputs):

        start_time = time.time()
        _ = lpsd.lpsd(x, fs=2000, object_return=True, pool=pool)
        times[cpu].append(time.time() - start_time)

    pool.close()
    pool.join()

In [ ]:
fig, ax = plt.subplots(figsize=(4,3), dpi=150)

for cpu, t in times.items():
    ax.loglog(N_values, t, label=f'CPU={cpu}')
ax.set_xlabel('Length of time series')
ax.set_ylabel('Computation time (s)')
ax.legend()
ax.grid(True, which='both', color='lightgray', linestyle='-', linewidth=0.5)
fig.tight_layout()


In [ ]:
for cpu in cpus:
	for i, t in enumerate(times[cpu]):
		print("{}, {}, {}".format(cpu, N_values[i], t))